In [129]:
#Importando bibliotecas

import numpy as np

import torch
import torch.nn as nn
    
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
import plotly.graph_objects as go

In [130]:
DATASET_NAME = 'UCI'

# CPC SSL Training

In [131]:
# Dados de Treino

data_path = Path('/workspaces/betania.silva/view_concatenated/UCI/train')

datas_x_train = []

data_y_train = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_train.append(x)

    y = data['standard activity code'].values

    data_y_train.append(y)


In [132]:
# Use a função shape para obter o shape de cada array na lista
formas = [arr.shape for arr in datas_x_train]

# Isso irá imprimir as formas (shapes) de cada array na lista
for i, forma in enumerate(formas):
    print(f"Array {i+1}: Shape {forma}")

Array 1: Shape (6, 8824)
Array 2: Shape (6, 8449)
Array 3: Shape (6, 7671)
Array 4: Shape (6, 8632)
Array 5: Shape (6, 7485)
Array 6: Shape (6, 7417)
Array 7: Shape (6, 9541)
Array 8: Shape (6, 7040)
Array 9: Shape (6, 8726)
Array 10: Shape (6, 8073)
Array 11: Shape (6, 8771)
Array 12: Shape (6, 9127)
Array 13: Shape (6, 7935)
Array 14: Shape (6, 7826)
Array 15: Shape (6, 8194)
Array 16: Shape (6, 9130)
Array 17: Shape (6, 7762)
Array 18: Shape (6, 7615)
Array 19: Shape (6, 8422)
Array 20: Shape (6, 8041)
Array 21: Shape (6, 7385)


In [133]:
# Dados de Teste

data_path = Path('/workspaces/betania.silva/view_concatenated/UCI/test')

datas_x_test = []

data_y_test = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_test.append(x)

    y = data['standard activity code'].values

    data_y_test.append(y)

In [134]:
x_train = datas_x_train
y_train = data_y_train
x_test = datas_x_test
y_test = data_y_test

In [135]:
datas_x_test

[array([[ 0.36540785,  0.79206858,  0.88780439, ..., -2.36982081,
         -3.2656766 , -2.03628733],
        [ 0.70760726,  1.03433844,  1.04630961, ..., -0.23616228,
         -0.08243935,  0.47299394],
        [-0.43062068, -0.74987103, -0.88142534, ...,  0.19780182,
         -1.84579955, -0.95706349],
        [ 0.00965204, -0.1060456 ,  0.00898792, ...,  0.53062411,
          1.09164325,  0.57848348],
        [ 0.01678245, -0.23185215, -0.0690665 , ..., -0.86830583,
         -0.61831919, -0.43782577],
        [-0.00475177, -0.02788059, -0.03397026, ..., -0.83912011,
         -0.8898552 , -0.44344236]]),
 array([[-0.17149129, -1.14195093,  1.06687611, ..., -0.7460073 ,
          0.60889507, -0.62005647],
        [ 1.06931844,  1.20454507,  1.35799043, ..., -1.34093223,
          0.25759907,  0.02948667],
        [ 0.97353149,  2.02599875,  1.18516601, ..., -0.2264351 ,
         -0.28342334,  0.39331783],
        [ 0.08992597, -0.14170856, -0.01781067, ...,  3.76012049,
          3.11

In [136]:
import torch
import torch.nn as nn

class CNNEncoder(nn.Module):
    def __init__(self, num_channels, kernel_size=3, dropout_rate=0.2, device: str = "cpu"):
        super(CNNEncoder, self).__init__()
        self.device = device

        self.conv1 = nn.Sequential(
            nn.Conv1d(num_channels, 32, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        x = x.to(self.device)  # Mova os dados para a GPU, se disponível
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.permute(0, 2, 1)
        print("CNN")
        print(x.size())

        # Aplana as duas primeiras dimensões
        x = x.reshape(-1, x.size(-1))

    # Verifica as dimensões do tensor aplanado
        print("Dimensões do tensor aplanado:", x.size())
        return x


In [137]:
import torch.nn as nn

class GRUEncoder(torch.nn.Module):
    def __init__(
        self,
        hidden_size: int = 256,
        in_channel: int = 128,
        encoding_size: int = 128,
        bidirectional: bool = True,
        device: str = "cpu",
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.in_channel = in_channel
        self.encoding_size = encoding_size
        self.bidirectional = bidirectional
        self.device = device

        # Defina uma camada GRU com duas camadas e 256 unidades em cada camada
        self.gru = nn.GRU(
            input_size=in_channel,
            hidden_size=256,  # 256 unidades em cada camada
            num_layers=2,      # Duas camadas GRU
            batch_first=True,
            bidirectional=bidirectional,
        ).to(device)

        self.nn = nn.Linear(
            self.hidden_size * (int(bidirectional) + 1), self.encoding_size
        ).to(device)

    def forward(self, x):
        print('Começando o forward:')
        print(x.size())
        # x = torch.squeeze(x)
        # #x = x.permute(0,2,1)
        # #print("permuta (0,2,1)")
        # past = torch.zeros(
        #     4,  # Duas camadas com bidirecionalidade resulta em 4 direções
        #     x.shape[0],
        #     256,  # Tamanho das unidades
        #     device=self.device,
        # )
        # print("Past")
        # print(past.size())
        out, _ = self.nn(
            x, past
        )   #out shape = [seq_len, batch_size, num_directions*hidden_size]
        encodings = self.nn(out[-1].squeeze(0))
        print(encodings.size())
        print('AQUI')
       # encodings = self.nn(out[-1].squeeze(0))
        print('exit')
      #  print(encodings.size())
        return encodings
        

In [138]:
class CPC(pl.LightningModule):
    def __init__(
        self,
        encoderCNN: torch.nn.Module,
        auto_regressor: torch.nn.Module,
        density_estimator: torch.nn.Module,
        #auto_regressor: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        window_size: int = 4,
        n_size: int = 5
    ):
        super().__init__()
        self.encoderCNN = encoderCNN.to(self.device)
        #self.encoder = encoder.to(self.device)
        self.density_estimator = density_estimator.to(self.device)
        self.auto_regressor = auto_regressor.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.window_size = window_size
        self.n_size = n_size
        self.training_step_losses = []


    def training_step(self, batch, batch_idx):
        assert len(batch) == 1, "Batch must be 1 sample only"
        sample = batch
        sample = sample.squeeze(0)
        rnd_t = np.random.randint(
            5 * self.window_size, sample.shape[-1] - 5 * self.window_size
        )
        sample = torch.tensor(
            sample[
                :,
                max(0, (rnd_t - 20 * self.window_size)) : min(
                    sample.shape[-1], rnd_t + 20 * self.window_size
                ),
            ]
        ).cpu()

        T = sample.shape[-1]
        
        windowed_sample = np.split(
            sample[:, : (T // self.window_size) * self.window_size],
            (T // self.window_size),
            -1,
        )
        windowed_sample = torch.tensor(np.stack(windowed_sample, 0), device=self.device)
        

        # Obtém as saídas do codificador CNN
        encodings = self.encoderCNN(windowed_sample)
        
        print("Encodings depois de passar pela cnn")
        
        print(encodings.size())

       # encodings = self.encoder(encodings_cnn)

        window_ind = torch.randint(2, len(encodings) - 2, size=(1,))

        print("window_ind")

        print(window_ind.size())

        print("AQUI1111")

        _, c_t = self.auto_regressor(
            encodings[max(0, window_ind[0] - 10) : window_ind[0] + 1]
        )
        
        print("pos probleMMMMMMMMMA")
        
        density_ratios = torch.bmm(
            encodings.unsqueeze(1),
            self.density_estimator(c_t.squeeze(1).squeeze(0)).expand_as(encodings).unsqueeze(-1),
        ).view(
            -1,
        )
        r = set(range(0, window_ind[0] - 2))
        r.update(set(range(window_ind[0] + 3, len(encodings))))
        rnd_n = np.random.choice(list(r), self.n_size)
        X_N = torch.cat(
            [density_ratios[rnd_n], density_ratios[window_ind[0] + 1].unsqueeze(0)], 0
        ).to(self.device)
        labels = torch.Tensor([len(X_N) - 1]).to(self.device)
        loss = torch.nn.CrossEntropyLoss()(X_N.view(1, -1), labels.long())
        self.training_step_losses.append(loss)
        return loss


    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
        
    def configure_optimizers(self):
        learnable_parameters = (
            list(self.density_estimator.parameters())
           # + list(self.encoder.parameters())
            + list(self.auto_regressor.parameters())
        )

        optimizer = torch.optim.Adam(
            learnable_parameters, lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer

In [139]:
num_channels = 6

encoding_size=128

# Cria o codificador
encoderCNN = CNNEncoder(num_channels, device='cuda')

# Cria o modelo autoregressivo com GRU
#encoder = GRUEncoder(in_channel=128, encoding_size=256, device='cpu')

density_estimator = torch.nn.Linear(encoding_size, encoding_size)

auto_regressor = torch.nn.GRU(input_size=128,
                          hidden_size=128,
                          num_layers=2,
                          bidirectional=False,
                          batch_first=True,
                          dropout=0.2)

#auto_regressor = GRUEncoder(in_channel=128, encoding_size=128, device='cuda')

cpc = CPC(encoderCNN, auto_regressor, density_estimator)


/home/betania_meta4/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [140]:
class SimpleDataset:
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
        else:
            return self.X[idx].astype(np.float32)
    
train_dataset = SimpleDataset(x_train)
test_dataset = SimpleDataset(x_test, y_test)
len(train_dataset), len(test_dataset)

(21, 6)

In [141]:
train_loader = DataLoader(train_dataset, batch_size=1, num_workers=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=10, shuffle=False)

In [142]:
trainer = pl.Trainer(max_epochs=5, accelerator="cuda", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [143]:
trainer.fit(cpc, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name              | Type       | Params
-------------------------------------------------
0 | encoderCNN        | CNNEncoder | 31.5 K
1 | density_estimator | Linear     | 16.5 K
2 | auto_regressor    | GRU        | 99.1 K
-------------------------------------------------
147 K     Trainable params
0         Non-trainable params
147 K     Total params
0.588     Total estimated model params size (MB)
/home/betania_meta4/.local/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/21 [00:00<?, ?it/s] CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0:   5%|▍         | 1/21 [00:00<00:00, 48.46it/s, v_num=221]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0:  10%|▉         | 2/21 [00:00<00:00, 59.51it/s, v_num=221]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0:  14%|█▍        | 3/21 [00:00<00:00, 57.92it/s, v_num=221]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Si

/tmp/ipykernel_331883/726988172.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(


CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0:  71%|███████▏  | 15/21 [00:00<00:00, 71.16it/s, v_num=221]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0:  76%|███████▌  | 16/21 [00:00<00:00, 70.92it/s, v_num=221]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0:  81%|████████  | 17/21 [00:00<00:00, 70.99it/s, v_num=221]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 0

CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 1:   0%|          | 0/21 [00:00<?, ?it/s, v_num=221]         CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 1:   5%|▍         | 1/21 [00:00<00:09,  2.22it/s, v_num=221, train_loss=1.790]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQUI1111
pos probleMMMMMMMMMA
Epoch 1:  10%|▉         | 2/21 [00:00<00:04,  4.35it/s, v_num=221, train_loss=1.790]CNN
torch.Size([40, 4, 128])
Dimensões do tensor aplanado: torch.Size([160, 128])
Encodings depois de passar pela cnn
torch.Size([160, 128])
window_ind
torch.Size([1])
AQU

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 21/21 [00:01<00:00, 20.63it/s, v_num=221, train_loss=1.810]


In [144]:
import torch

# Suponha que você tenha um tensor tridimensional com dimensões [batch_size, dim1, dim2]
tensor_tridimensional = torch.randn(4, 5, 6)

# Aplana as duas primeiras dimensões
tensor_aplanado = tensor_tridimensional.view(-1, tensor_tridimensional.size(-1))

# Verifica as dimensões do tensor aplanado
print("Dimensões do tensor aplanado:", tensor_aplanado.size())

Dimensões do tensor aplanado: torch.Size([20, 6])


In [145]:
# # Checking if there are weights saved from previous training, else it trains from scratch
# try:
#     encoder.load_state_dict(torch.load(f'./weights/{DATASET_NAME}_encoder.pt'))
# except:
#     torch.save(encoder.state_dict(), f'./weights/{DATASET_NAME}_encoder.pt')

## CPC Fine-Tuning

We are going to fine-tune the CPC model on the downstream task of classification. We will use the same dataset and re-use the same encoder.

In [146]:
# from typing import Any, Optional

# from torchmetrics.functional import accuracy

# class StateClassifier(torch.nn.Module):
#     def __init__(self, input_size: int = 10, n_classes: int = 7):
#         super(StateClassifier, self).__init__()
#         self.input_size = input_size
#         self.n_classes = 7
#         self.normalize = torch.nn.BatchNorm1d(self.input_size)
#         self.nn = torch.nn.Linear(self.input_size, self.n_classes)
#         torch.nn.init.xavier_uniform_(self.nn.weight)

#     def forward(self, x):
#         x = self.normalize(x)
#         logits = self.nn(x)
#         return logits


# class CPC_Classifier(pl.LightningModule):
#     def __init__(
#         self,
#         encoderCNN: torch.nn.Module,
#         encoder: torch.nn.Module,
#         classifier: torch.nn.Module,
#         lr: float = 1e-3,
#         weight_decay: float = 0.0,
#         task_class: str = "multiclass",
#         num_classes: int = 7
#     ):
#         super().__init__()
#         self.encoderCNN = encoderCNN.to('cpu')
#         self.encoder = encoder.to('cpu')
#         self.classifier = classifier.to('cpu')
#         self.learning_rate = lr
#         self.weight_decay = weight_decay
#         self.training_step_losses = []
#         self.validation_step_losses = []
#         self.loss_function = torch.nn.CrossEntropyLoss()
#         self.task_class = task_class
#         self.num_classes = 7
        
#     def configure_optimizers(self) -> Any:
#         optimizer = torch.optim.Adam(
#             self.classifier.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
#         )
#         return optimizer
    
#         # def configure_optimizers(self):
#         # learnable_parameters = (
#         #     list(self.encoderCNN.parameters())
#         #     + list(self.encoder.parameters())
#         #     + list(self.classifier.parameters())
#         # )

#         # optimizer = torch.optim.Adam(
#         #     learnable_parameters, lr=self.learning_rate, weight_decay=self.weight_decay
#         # )
#         # return optimizer
    
#     def forward(self, x):
#         encodings_cnn = self.encoderCNN(x)
#         encodings = self.encoder(encodings_cnn)
#         #predictions = self.classifier(encodings)
#         #return predictions
#         return encodings
    
#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         predictions = self.forward(x)
#         loss = self.loss_function(predictions, y.long())
#         self.training_step_losses.append(loss)
#         return loss
    
#     def on_train_epoch_end(self) -> None:
#         # do something with all training_step outputs, for example:
#         epoch_mean = torch.stack(self.training_step_losses).mean()
#         self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
#         # free up the memory
#         self.training_step_losses.clear()
    
#     def validation_step(self, batch, batch_idx):
#         loss, acc = self._shared_eval_step(batch, batch_idx)
#         self.validation_step_losses.append(loss)
#         metrics = {"val_acc": acc, "val_loss": loss}
#         self.log_dict(metrics)
#         return metrics
    
#     def test_step(self, batch, batch_idx):
#         loss, acc = self._shared_eval_step(batch, batch_idx)
#         metrics = {"test_acc": acc, "test_loss": loss}
#         self.log_dict(metrics)
#         return metrics

        
#     def on_validation_epoch_end(self) -> None:
#         # do something with all training_step outputs, for example:
#         epoch_mean = torch.stack(self.validation_step_losses).mean()
#         self.log("val_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
#         # free up the memory
#         self.validation_step_losses.clear()

#     def _shared_eval_step(self, batch, batch_idx):
#         x, y = batch
#         predictions = self.forward(x)
#         loss = self.loss_function(predictions, y.long())
#         acc = accuracy(torch.argmax(predictions, dim=1), y.long(), task=self.task_class, num_classes=self.num_classes)
#         return loss, acc

In [147]:
class EncoderClassifier(nn.Module):
    def __init__(self, num_channels, kernel_size=3, dropout_rate=0.2, device: str = "cpu"):
        super(EncoderClassifier, self).__init__()
        self.device = device

        self.conv1 = nn.Sequential(
            nn.Conv1d(num_channels, 32, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        x = x.to(self.device)  # Mova os dados para a GPU, se disponível
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.permute(0, 2, 1)
        print("CNN")
        print(x.size())

        # Aplana as duas primeiras dimensões
        x = x.reshape(-1, x.size(-1))

    # Verifica as dimensões do tensor aplanado
       # print("Dimensões do tensor aplanado:", x.size())
        return x

In [148]:
from typing import Any, Optional

from torchmetrics.functional import accuracy

# Classificando com uma MLP

class StateClassifier(torch.nn.Module):
    def __init__(self, input_size: int= 128, hidden_size1= 64, hidden_size2=64, n_classes= 7, dropout_prob= 0):
        super(StateClassifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(p=dropout_prob)
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size2, n_classes),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        print('st1_in:', x.size())
        out = self.layer1(x)
        print('st2_in:', x.size())
        out = self.layer2(out)
        print('stdp_in:', x.size())
        out = self.dropout(out)
        print('stout_in:', x.size())
        out = self.output_layer(out)
        print('out_:', x.size())
        return out

class CPC_Classifier(pl.LightningModule):
    def __init__(
        self,
        encoderCNN: torch.nn.Module,
        classifier: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        task_class: str = "multiclass",
        n_classes: int = 7
    ):
        super().__init__()
        self.encoderCNN = encoderCNN.to(self.device)
        self.classifier = classifier.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.training_step_losses = []
        self.validation_step_losses = []
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.task_class = task_class
        self.n_classes = n_classes 
        
    def configure_optimizers(self) -> Any:
        optimizer = torch.optim.Adam(
            self.classifier.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer
    
    def forward(self, x):
        encodings = self.encoderCNN(x)
        print('saindo do encoding:', encodings.size())
        predictions = self.classifier(encodings)
        print('pred_outed:', predictions)
        return predictions
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        self.training_step_losses.append(loss)
        return loss
    
    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        self.validation_step_losses.append(loss)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics
    
    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

        
    def on_validation_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.validation_step_losses).mean()
        self.log("val_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.validation_step_losses.clear()

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        acc = accuracy(torch.argmax(predictions, dim=1), y.long(), task=self.task_class, num_classes=self.num_classes)
        return loss, acc

In [149]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 60

# Caminho dos dados

data_path_train = Path('/workspaces/betania.silva/data/standartized_balanced/UCI/train.csv')

data_path_validation = Path('/workspaces/betania.silva/data/standartized_balanced/UCI/validation.csv')

data_path_test = Path('/workspaces/betania.silva/data/standartized_balanced/UCI/test.csv')

# Train

x_train = pd.read_csv(data_path_train)

x_train = x_train.iloc[:, :360]
    
x_train = x_train.astype(np.float32)

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train = y_train.astype(np.float32)

tensor_x = torch.Tensor(np.array(x_train))

tensor_y = torch.Tensor(np.array(y_train))

original_dim = tensor_x.shape[0]

input_shape = (original_dim, 6, 60)

tensor_x = tensor_x.reshape(input_shape)

dataset_train= TensorDataset(tensor_x,tensor_y)

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)

# Validation

x_validation = pd.read_csv(data_path_validation)

x_validation = x_validation.iloc[:, :360]

x_validation = x_validation.astype(np.float32)

y_validation = pd.read_csv(data_path_validation)

y_validation = y_validation.iloc[:, -1]

y_validation = y_validation.astype(np.float32)

tensor_x_val = torch.Tensor(np.array(x_validation))

tensor_y_val = torch.Tensor(np.array(y_validation))

original_dim_val = tensor_x_val.shape[0]

tensor_x_val = tensor_x_val.reshape(original_dim_val, 6, 60)

dataset_val= TensorDataset(tensor_x_val,tensor_y_val)

dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, drop_last = True)

# Test

x_test = pd.read_csv(data_path_test)

x_test = x_test.iloc[:, :360]

x_test = x_test.astype(np.float32)

y_test = pd.read_csv(data_path_test)

y_test = y_test.iloc[:, -1]

y_test = y_test.astype(np.float32)

tensor_x_test = torch.Tensor(np.array(x_test))

tensor_y_test = torch.Tensor(np.array(y_test))

original_dim_test = tensor_x_test.shape[0]

tensor_x_test = tensor_x_test.reshape((original_dim_test, 6, 60))

dataset_test= TensorDataset(tensor_x_test,tensor_y_test)

dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)


In [150]:
original_dim_test = tensor_x_test.shape[0]

original_dim_test

690

In [151]:
data_path_train = Path('/workspaces/betania.silva/data/standartized_balanced/UCI/train.csv')

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train.unique()

array([2, 3, 4, 0, 1])

In [152]:
x_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-50,gyro-z-51,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59
0,4.339089,-1.167666,-3.694124,-3.991698,-3.144751,-4.243802,-0.346311,4.784375,5.407283,2.445792,...,0.022887,-0.159737,-0.375233,0.051135,0.921111,0.278494,-0.133034,-0.033831,-0.145661,-0.039100
1,-1.999586,-1.566372,-1.517025,-2.713303,0.712069,2.309685,3.010014,3.875827,1.239790,-0.457387,...,-0.684964,-0.555177,-0.076758,0.106422,0.348305,0.280755,0.141009,-0.206322,0.010000,0.119462
2,-0.876184,-1.637183,-0.968435,-4.894519,-2.211239,-0.815775,1.349346,0.910528,1.072714,2.240573,...,0.064544,0.164004,0.180907,-0.684902,-0.689362,0.008271,0.051843,0.089196,-0.178239,0.031978
3,1.649148,-1.854505,3.019927,3.966432,-0.782214,-2.535543,-3.642630,-2.326545,-2.164334,-0.037000,...,-0.264074,0.088056,0.465905,0.422788,-0.038715,-0.766346,0.376857,-0.127370,0.074219,0.437026
4,3.431428,0.846879,0.397590,1.620648,-2.014615,-3.534210,-1.916457,-1.600943,-1.037111,0.433378,...,0.246871,0.168610,0.257335,0.179625,0.129522,0.167631,-0.061982,0.597766,-0.117428,-0.170377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,-0.055121,-0.013841,-0.015698,-0.032567,-0.020638,0.020384,0.029549,0.035703,-0.009456,-0.038961,...,-0.005325,-0.000865,-0.006344,-0.000562,-0.011373,-0.000161,0.007937,-0.001771,-0.000563,-0.000628
2416,-0.015348,-0.050589,-0.000972,0.005511,0.010061,0.016410,-0.041210,0.058844,-0.025887,-0.011735,...,0.018803,0.016867,0.012139,0.007154,0.008134,-0.001346,-0.000821,0.003322,-0.006642,-0.012614
2417,-0.003522,0.015157,0.029197,-0.022950,-0.015183,-0.001276,0.010532,-0.017849,-0.042912,0.004876,...,0.001644,0.000004,0.003882,0.006880,0.001106,0.005042,0.008657,0.002099,0.008879,0.005425
2418,0.162229,0.062337,-0.228693,-0.053321,0.088912,0.067402,-0.140253,-0.095661,0.138672,-0.058650,...,0.007322,0.005421,0.004941,-0.003199,-0.009513,-0.007301,-0.003157,-0.001795,0.006901,-0.002802


In [153]:
# shape dataloader

for i, (x, y) in enumerate(dataloader_train):
    print(f"Batch {i} shape: {x.shape}")

for i, (x, y) in enumerate(dataloader_val):
    print(f"Batch {i} shape: {y.shape}")
    

Batch 0 shape: torch.Size([60, 6, 60])
Batch 1 shape: torch.Size([60, 6, 60])
Batch 2 shape: torch.Size([60, 6, 60])
Batch 3 shape: torch.Size([60, 6, 60])
Batch 4 shape: torch.Size([60, 6, 60])
Batch 5 shape: torch.Size([60, 6, 60])
Batch 6 shape: torch.Size([60, 6, 60])
Batch 7 shape: torch.Size([60, 6, 60])
Batch 8 shape: torch.Size([60, 6, 60])
Batch 9 shape: torch.Size([60, 6, 60])
Batch 10 shape: torch.Size([60, 6, 60])
Batch 11 shape: torch.Size([60, 6, 60])
Batch 12 shape: torch.Size([60, 6, 60])
Batch 13 shape: torch.Size([60, 6, 60])
Batch 14 shape: torch.Size([60, 6, 60])
Batch 15 shape: torch.Size([60, 6, 60])
Batch 16 shape: torch.Size([60, 6, 60])
Batch 17 shape: torch.Size([60, 6, 60])
Batch 18 shape: torch.Size([60, 6, 60])
Batch 19 shape: torch.Size([60, 6, 60])
Batch 20 shape: torch.Size([60, 6, 60])
Batch 21 shape: torch.Size([60, 6, 60])
Batch 22 shape: torch.Size([60, 6, 60])
Batch 23 shape: torch.Size([60, 6, 60])
Batch 24 shape: torch.Size([60, 6, 60])
Batch 25 s

In [154]:
encoding_size = 128

n_classes = 7

num_channels = 6

CNNclassifier = EncoderClassifier(num_channels=6, device='cuda')

classifier = StateClassifier(input_size=encoding_size, n_classes=n_classes, dropout_prob=0, hidden_size1= 64, hidden_size2=32)

cpc_classifier = CPC_Classifier(CNNclassifier, classifier)


In [155]:
# cpc_classifier = cpc_classifier.to('cpu')

# xtest = cpc_classifier.forward(tensor_x_test.to('cpu'))

# xtest = xtest.detach().numpy()

# xtrain = cpc_classifier.forward(tensor_x.to('cpu'))

# xtrain = xtrain.detach().numpy()

# ytrain = tensor_y.detach().numpy()

# ytest = tensor_y_test.detach().numpy()


In [156]:
# from sklearn.ensemble import RandomForestClassifier

# from sklearn.metrics import accuracy_score

# random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# random_forest_model.fit(xtrain, ytrain)

# # Faça previsões no conjunto de teste
# predictions = random_forest_model.predict(xtest)

# # Avalie o desempenho do modelo usando métricas, como a precisão
# accuracy = accuracy_score(ytest, predictions)

# print(f'A precisão do modelo no conjunto de teste é: {accuracy}')

In [157]:
# # KNN

# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score

# # Criar o modelo KNN
# knn_model = KNeighborsClassifier(n_neighbors=5)  # Pode ajustar o número de vizinhos conforme necessário

# # Treinar o modelo
# knn_model.fit(xtrain, ytrain)

# # Fazer previsões no conjunto de teste
# predictions_knn = knn_model.predict(xtest)

# # Avaliar o desempenho do modelo usando métricas, como a precisão
# accuracy_knn = accuracy_score(ytest, predictions_knn)

# print("Accuracy (KNN):", accuracy_knn)


In [158]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# # Criar o modelo SVM
# svm_model = SVC(kernel='linear', C=1.0)  # Pode ajustar o kernel e C conforme necessário

# # Treinar o modelo
# svm_model.fit(xtrain, ytrain)

# # Fazer previsões no conjunto de teste
# predictions_svm = svm_model.predict(xtest)

# # Avaliar o desempenho do modelo usando métricas, como a precisão
# accuracy_svm = accuracy_score(ytest, predictions_svm)

# print("Accuracy (SVM):", accuracy_svm)


In [159]:
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [160]:
trainer.fit(cpc_classifier, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type              | Params
----------------------------------------------------
0 | encoderCNN    | EncoderClassifier | 31.5 K
1 | classifier    | StateClassifier   | 10.6 K
2 | loss_function | CrossEntropyLoss  | 0     
----------------------------------------------------
42.1 K    Trainable params
0         Non-trainable params
42.1 K    Total params
0.168     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]CNN
torch.Size([60, 60, 128])
saindo do encoding: torch.Size([3600, 128])
st1_in: torch.Size([3600, 128])
st2_in: torch.Size([3600, 128])
stdp_in: torch.Size([3600, 128])
stout_in: torch.Size([3600, 128])
out_: torch.Size([3600, 128])
pred_outed: tensor([[0.1469, 0.1409, 0.1192,  ..., 0.1437, 0.1392, 0.1447],
        [0.1465, 0.1410, 0.1195,  ..., 0.1438, 0.1392, 0.1445],
        [0.1471, 0.1408, 0.1193,  ..., 0.1439, 0.1390, 0.1446],
        ...,
        [0.1468, 0.1408, 0.1191,  ..., 0.1436, 0.1391, 0.1448],
        [0.1468, 0.1408, 0.1191,  ..., 0.1437, 0.1391, 0.1448],
        [0.1465, 0.1413, 0.1193,  ..., 0.1435, 0.1391, 0.1445]],
       device='cuda:0')


/home/betania_meta4/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/betania_meta4/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


ValueError: Expected input batch_size (3600) to match target batch_size (60).

## Classification evaluation

In [ ]:
trainer.test(cpc_classifier, dataloader_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 21/21 [00:00<00:00, 82.85it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6592261791229248
        test_loss           1.4835978746414185
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.6592261791229248, 'test_loss': 1.4835978746414185}]